In [1]:
%run Signed_fundamental_domain_Espinoza.ipynb
%run Log_gamma_Ruijsenaars.ipynb
%run Intersect_lattice_with_cone.ipynb

SIMPLE SUB-FUNCTIONS

In [2]:
def get_key(my_dict, value):
    """Return the dictionary key of the given value."""
    key_list = list(my_dict.keys())
    val_list = list(my_dict.values())
    
    position = val_list.index(value)
    return key_list[position]

In [3]:
def my_arg(z):
    """Calculate the argument of the complex number z with branch cut in [0, 2pi)."""
    # recall: in Sage, arguments of complex numbers lie in (-pi, pi].
    argument = arg(z)
    if argument < 0:
        argument += 2*pi
    return float(argument)

In [4]:
def my_log(z):
    """Calculate the logarithm of the complex number z with branch cut along the positive reals."""
    return float(log(abs(z))) + I*float(my_arg(z))

COMPUTE SCRIPT-LR functions

In [5]:
def sum_Shintani_over_xvecs(r, A, list_xvecs, permutations_of_r):
    """Sum the Shintani zeta at 0 over the given xvecs."""
    sum = 0
    for xvec in list_xvecs:
        Barnes = Shintani_zeta_at_0(r, A, xvec, permutations_of_r)
        sum += Barnes
    return sum

In [6]:
def sum_Barnes_over_xvecs(r, columnofA, list_xvecs, permutations_of_r):
    """Sum the Barnes zeta at 0 over the given xvecs."""
    return sum_Shintani_over_xvecs(r, [columnofA], list_xvecs, permutations_of_r)

In [7]:
def compare_Barnes_over_xvecs(r, columnofA, list_xvecs, Bern, permutations_of_r):
    """Sum the Barnes zeta at 0 over the given xvecs."""
    for xvec in list_xvecs:
        Barnes2 = Shintani_Barnes_zeta_at_0(r, columnofA, xvec, permutations_of_r)
        fieldx = dot_product(xvec, columnofA) # dot product of xvec and columnofA
        Barnes1 = Barnes_zeta_at_0(r, columnofA, fieldx, Bern)
        difference = abs(Barnes1 - Barnes2) 
        if difference > 10^(-5):
            print('for xvec =', xvec, 'the Barnes are different by', difference)
    return sum

In [8]:
def compute_rhos(domain, colA, xvecs, xvecs_prime, Berndict):
    """Compute the rhos, and return the results in a dictionary.
    Check that the rhos are integers, print a warning otherwise."""

    field = domain.field
    cones = domain.cones
    cone_keys = cones.keys()
    r = field.n
    permutations_of_r = get_permutations(r)
    zero_cutoff = sqrt(domain.zero_cutoff)
    
    print('Computing rhos.', end = '\n\n')
    rhos = dict([]) # dictionary indexed by cones
    embedding = field.complex_embeddings[0]  # we've checked that we can put any embedding here

    sum_rhos = 0
    for mu in cone_keys: # each rho is a sum of Barnes zeta functions over the cone
        cone = cones[mu]
        columnofA = colA[mu][embedding]
        # A = list(colA[mu].values())  # using all of A multiplied the rhos by n... 

        # import pdb
        # pdb.set_trace()
        
        twisted_sum_Shintani = 0
        twisted_sum_Shintani += -sum_Barnes_over_xvecs(r, columnofA, xvecs[mu], permutations_of_r)
        twisted_sum_Shintani += ell*sum_Barnes_over_xvecs(r, columnofA, xvecs_prime[mu], permutations_of_r)
        # twisted_sum_Shintani += -sum_Shintani_over_xvecs(r, A, xvecs[mu], permutations_of_r)
        # twisted_sum_Shintani += ell*sum_Shintani_over_xvecs(r, A, xvecs_prime[mu], permutations_of_r)

        
        if not abs(twisted_sum_Shintani.imag()) < zero_cutoff:
            print('for mu =', mu, ', twisted_sum_Shintani is not real!')
        else:
            twisted_sum_Shintani = twisted_sum_Shintani.real()
            if not (twisted_sum_Shintani - round(twisted_sum_Shintani)) < zero_cutoff:
                print('for mu =', mu, ', twisted_sum_Shintani is real, but not integral!')
            else:
                twisted_sum_Shintani = round(twisted_sum_Shintani)
                
        rhos[mu] = domain.orientations[mu]*twisted_sum_Shintani
        
        sum_rhos += rhos[mu]
        print('mu =', mu, '\t rho =', rhos[mu])
    print('sum of rhos =', sum_rhos)
    print()
    return rhos

In [9]:
def compute_matrices_of_cones(domain): # ToDO: add to domain dataclass?
    """Compute matrices of cones, and return in a dictionary.
    Assume that domain already contains chosen rotations."""
    field = domain.field
    cones = domain.cones
    cone_keys = cones.keys()
    complex_embed = field.complex_embeddings[0]
    conj_complex_embed = field.complex_embeddings[1]
    rotations = domain.rotations
    zero_cutoff = sqrt(domain.zero_cutoff)
    
    colA = dict([]) # will be indexed by cones
    
    print('Computing matrices of cones.')
    for mu in cone_keys:
        cone = cones[mu] # for each cone: compute A (by column) and compute t's
        colA[mu] = dict([]) # will be indexed by embeddings
        
        # store columns of real embeddings
        for real_place in field.real_places: 
            colA[mu][real_place] = embed(cone, real_place)
        
        # get columns of complex embeddings
        column = embed(cone, complex_embed)
        conj_column = embed(cone, conj_complex_embed)
        
        # rotate columns of complex embeddings so real parts > 0
        rotation = get_decimals(rotations[mu])
        print('mu =', mu, '\t rotation =', rotations[mu], '=', rotation)
        for j in range(0, len(column)):
            column[j] *= rotation
            conj_column[j] *= rotation^(-1)
            # check expected conjugacy of columns
            if not (column[j].conjugate() - conj_column[j] < zero_cutoff):
                print('Columns are not conjugate...?!')
            # check positive real part
            if not (column[j].real() > 0):
                print('Columns do not have positive real part...?!')
        
        # store rotated columns
        colA[mu][complex_embed] = column
        colA[mu][field.complex_embeddings[1]] = conj_column
    print()
    return colA

In [10]:
def check_partial_zeta(domain, fraka, frakc, colA = None, xvecs = None, xvecs_prime = None):
    field = domain.field
    r = field.n
    permutations_of_r = get_permutations(r)
    sum = 0
    zero_cutoff = domain.zero_cutoff

    # choose rotations if necessary
    if domain.rotations == dict([]):
        domain.choose_rotations(weighted = False)
    rotations = domain.rotations
    
    if colA == None:
        domain.scale_cone_gens([fraka, field.f])
        colA = compute_matrices_of_cones(domain)
    if xvecs == None:
        xvecs = get_xvecs(domain, fraka)
    if xvecs_prime == None:
        domain.scale_cone_gens([fraka, frakc, field.f])
        xvecs_prime = get_xvecs(domain, fraka*frakc)

    for mu in domain.cones.keys(): # compute as sum over the cones of Shintani zeta functions
        
        # setup matrix A
        # A = [colA[mu][embedding]] 
        A = []
        for embedding in field.real_places:
            A.append(colA[mu][embedding])
        for embedding in field.complex_embeddings:
            A.append(colA[mu][embedding]) # TODO: try this when multiplying by l also

        twisted_sum_Shintani = 0
        for xvec in xvecs[mu]: # calculate sum of Shintani zeta functions
            Shintani = Shintani_zeta_at_0(r, A, xvec, permutations_of_r)
            twisted_sum_Shintani += -Shintani
        for xvec_prime in xvecs_prime[mu]: 
            Shintani = Shintani_zeta_at_0(r, A, xvec_prime, permutations_of_r)
            twisted_sum_Shintani += ell*Shintani
        
        sum += domain.orientations[mu]*twisted_sum_Shintani # record with appropriate orientation

    if not abs(sum) < zero_cutoff:
        print('!!! Partial zeta function does not equal zero at s = 0 !!! \n')
    else:
        print('Checked that the partial zeta function equals 0 at s = 0. \n')

In [11]:
def compute_scriptLsRs(domain, fraka, frakc, threshold = 10, M = 50, checking = False):
    """Return script L's and R's for the given domain, integral ideal fraka, and smoothing ideal frakc.
    Scale the generators of the cones of domain so that they are primitive elements of fraka*frakc."""
    # if checking: # TODO: perform some timed experiments of log_gamma
        # from statistics import mean # the import raised an error because it has a different 'r'

    import sympy as sym
    
    # setup variables
    field = domain.field
    cones = domain.cones
    cone_keys = cones.keys()
    ell = frakc.norm()
    complex_embed = field.complex_embeddings[0]
    complex_embed_conj = field.complex_embeddings[1]

    # choose rotations if necessary
    if domain.rotations == dict([]):
        domain.choose_rotations(weighted = False)
    rotations = domain.rotations

    print('Computing the R_p sets.\n')

    # scale cone generators so that each one is a primitive element of fraka*f
    domain.scale_cone_gens([fraka, field.f])
    # compute matrices of cone
    colA = compute_matrices_of_cones(domain)
    

    # scale cone generators so that each one is a primitive element of fraka*frakc*f
    domain.scale_cone_gens([fraka, frakc, field.f])
    # compute the prime-to-ell xvecs
    xvecs = get_xvecs(domain, fraka) # dictionary indexed by cones
    # compute the ell-xvecs
    xvecs_prime = get_xvecs(domain, fraka*frakc) # dictionary indexed by cones
    
    # setup dictionaries
    scriptLs = dict([]) # indexed by embeddings
    for embedding in field.embeddings:
        scriptLs[embedding] = 0
    scriptRs = dict([]) # indexed by complex embeddings
    for embedding in field.complex_embeddings:
        scriptRs[embedding] = 0
    Berndict = compute_Bernoullis(field.n, M)

    # check that the partial zeta function equals zero at s = 0
    if checking:
        check_partial_zeta(domain, fraka, frakc, colA, xvecs, xvecs_prime)

    # compute rhos
    rhos = compute_rhos(domain, colA, xvecs, xvecs_prime, Berndict)
    
    # compute script R's, indexed by complex embeddings
    for mu in cone_keys:
        rotation = rotations[mu]
        print('rotation =', rotation)
        scriptRs[complex_embed] += sym.simplify(log(rotation))*rhos[mu]
        scriptRs[complex_embed_conj] += sym.simplify(log(rotation^(-1)))*rhos[mu]

    # compute script L's (indexed by embeddings)
    print('Computing script Ls.') 
    # scale cone generators so that each one is a primitive element of fraka*f
    domain.scale_cone_gens([fraka, field.f])

    list_of_zs = []
    list_of_zprimes = []
    
    for mu in cone_keys: # each script L is a sum over the cones of log gamma functions 
        print('Computing for cone with mu =', mu, '\n')
        cone_gens = cones[mu]
        cone_gens_prime = [ell*value for value in cone_gens]
        orientation = domain.orientations[mu]
       
        for embedding in field.embeddings: # for each embedding, add contribution of cone to all scriptLs
            
            print('Computing for embedding with embedding(theta) =', embedding(field.theta).n(digits = 3), '\n')
            # get rotation
            if embedding == complex_embed:
                rotation = rotations[mu]
            elif embedding == complex_embed_conj:
                if not checking: # if not checking, skip conjugate
                    continue
                else:  # if checking, compute conjugate pairs by hand
                    rotation = rotations[mu]^(-1) 
            else:  # real rotations are 1
                rotation = 1
              
            # compute data for xvecs
            columnofA = colA[mu][embedding]
            
            cone_approx_data = logGammaApproxData(cone_gens, columnofA, threshold = threshold, M = M)
            recursive_log_gamma(cone_approx_data, xvecs[mu], embedding)

            for xvec in xvecs[mu]:
                z = dot_product(columnofA, xvec)           
                list_of_zs.append(n(abs(z), digits = 5))
            
            # compute data for xvecs_prime
            columnofA_prime = [ell*value for value in columnofA]
            cone_approx_data_prime = logGammaApproxData(cone_gens_prime, columnofA_prime, threshold = threshold, M = M)
            recursive_log_gamma(cone_approx_data_prime, xvecs_prime[mu], embedding)

            for xvec_prime in xvecs_prime[mu]:
                z = dot_product(columnofA_prime, xvec_prime)
                list_of_zprimes.append(n(abs(z), digits = 5))
            
            # add contribution
            scriptLs[embedding] += orientation*scriptL_cone_contribution(xvecs[mu], cone_approx_data, xvecs_prime[mu], cone_approx_data_prime, rotation, embedding, ell)
    
    if not checking:
        scriptLs[complex_embed_conj] = scriptLs[complex_embed].conjugate() # use known conjugacy

    # print results
    for embedding in field.embeddings:
        scriptLs[embedding] = get_decimals(scriptLs[embedding]) 
    print('scriptLs =', scriptLs.values())
    print('scriptRs =', scriptRs.values())
    print()

    # plot counts of z and zprimes
    
    import matplotlib.pyplot as plt
    import numpy as np
    
    # Creating a stacked histogram
    plt.hist([list_of_zs, list_of_zprimes], bins = 10, stacked = True, color = ['cyan', 'purple'], edgecolor = 'black')
     
    # Plotting
    plt.xlabel('Abs value of z')
    plt.ylabel('Frequency')
    plt.title('Frequency of Absolute Values of z')
    plt.legend(['Abs vals of z', 'Abs vals of zprime'])
    name = 'AbsVal_Graphs/Abs vals of z\'s for ' + field.source
    plt.savefig(name + '.png')
    
    return scriptLs, scriptRs

In [12]:
def scriptL_cone_contribution(list_xvecs, cone_approx_data, list_xvecs_prime, cone_approx_data_prime, dec_rotation, embedding, ell):
    # sum over xvecs       
    contribution = 0
    for xvec in list_xvecs:
       
        term = -log_gamma_from_data(cone_approx_data, xvec, dec_rotation, embedding)
        # pdb.set_trace()
        contribution += term
        # print('term =', term.n(digits = 6), 'running total =', contribution.n(digits = 6))
    
    # sum over xvecs_prime
    for xvec_prime in list_xvecs_prime:
        term = ell*log_gamma_from_data(cone_approx_data_prime, xvec_prime, dec_rotation, embedding)
        # pdb.set_trace()
        contribution += term
        # print('term =', term.n(digits = 6), 'running total =', contribution.n(digits = 6))
    
    return contribution

In [13]:
def equiv_mod_2piI(LHS, RHS, zero_cutoff = 10^(-5)):
    """Return true if LHS - RHS is equivalent to 0 modulo 2*pi*I*bbZ."""
    diff = LHS - RHS
    if not abs(diff.real()) < zero_cutoff: # expect difference is totally imaginary
        return False
    scaled_diff = diff.imag()/(2*pi)
    if not abs(scaled_diff - round(scaled_diff)) < zero_cutoff: # expect scaled difference is an integer
        return False
    return True

In [14]:
def check_embedding_LR(field, scriptLs, scriptRs):
    """Check the expected relationships (by embedding) of the script Ls and Rs."""
    complex_embed = field.complex_embeddings[0]
    complex_embed_conj = field.complex_embeddings[1]
    zero_cutoff = 10^(-5)
    
    real = True
    for embedding in field.real_places: # for each real place
        if not scriptLs[embedding].imag() == 0:
            real = False
    print('scriptLs of real embeddings are real:', real)

    complexL = True
    L1 = scriptLs[complex_embed]
    L2 = scriptLs[complex_embed_conj]
    if not abs(L1 - L2.conjugate()) < zero_cutoff:
        complexL = False
    print('scriptLs of complex embeddings are conjugate:', complexL)

    R1 = scriptRs[complex_embed]
    R2 = scriptRs[complex_embed_conj]
    complexRdiff = equiv_mod_2piI(R1, R2.conjugate(), zero_cutoff)
    print('scriptRs of complex embeddings are conjugate modulo 2*pi*I*Z:', complexRdiff)

    complexRsum = True
    sumR = R1 + R2
    if not abs(sumR.real()) < zero_cutoff: # expect sum is totally imaginary
        complexRsum = False
    scaled_sumR = sumR.imag()/(2*pi)
    if not abs(scaled_sumR - round(scaled_sumR)) < zero_cutoff: # expect scaled sum is an integer
        complexRsum = False
    print('scriptRs of complex embeddings sum to element of 2*pi*I*Z:', complexRsum)

In [15]:
def check_rotation_invariance(domain):
    domain.choose_rotations(weighted = True)
    (scriptLs2, scriptRs2) = compute_scriptLsRs(domain, fraka, frakc, threshold = 3, M = 7, checking = False)
    domain.choose_rotations(weighted = False)
    (scriptLs, scriptRs) = compute_scriptLsRs(domain, fraka, frakc, threshold = 3, M = 7, checking = False)

    print()
    print('scriptLs =', scriptLs.values())
    print('scriptLs2 =', scriptLs2.values())
    print()
    print('scriptRs =', scriptRs.values())
    print('scriptRs2 =', scriptRs2.values())
    print()
    
    for embedding in domain.field.complex_embeddings:
        LHS = scriptLs[embedding] + scriptRs[embedding]
        RHS = scriptLs2[embedding] + scriptRs2[embedding]
        equivalence = equiv_mod_2piI(LHS, RHS, domain.zero_cutoff)
        print('scriptL + scriptR is independent of T mod 2*pi*I*bbZ:', equivalence) 

    print()
    
    Stark = compute_Stark_from_LRs(domain.field, scriptLs, scriptRs)
    print('Stark with averages rotation = \t', Stark)

    Stark2 = compute_Stark_from_LRs(domain.field, scriptLs2, scriptRs2)
    print('Stark with weighted rotation = \t', Stark2)

COMPUTE STARK from conjectural formula

In [16]:
import numpy
def compute_Stark_from_LRs(field, scriptLs, scriptRs, checking = False):
    """Compute and return the conjectural Stark unit from the script Ls and scripts Rs."""    
    n = field.n
    complex_embed = field.complex_embeddings[0]
    complex_embed_conj = field.complex_embeddings[1]
    embeddings = field.embeddings
    units = field.units[1:]

    get_index = dict([])
    index = 0
    for embedding in embeddings: # store indices of columns
        get_index[embedding] = index
        index += 1
        
    M = [] # matrix of logarithms of units
    for unit in units: # index rows by (n-2) units
        row = []
        for embedding in embeddings: # index columns by n embeddings
            row.append(log(embedding(unit)))
        M.append(row)

    # store determinants
    def get_det(M, index1, index2):
        """Return determinant of square matrix obtained from M by deleting the columns at index1 and index2."""
        return det(matrix(numpy.delete(M, [index1, index2], 1)))
    Dets = dict([])
    c_index = get_index[complex_embed]
    for embedding in field.real_places: # determinant when remove column of real embedding
        m = get_index[embedding]
        Dets[embedding] = get_det(M, m, c_index) # delete m^th and n^th columns, then determinant
    cconj_index = get_index[complex_embed_conj]
    Dets[complex_embed_conj] = get_det(M, cconj_index, c_index) # delete complex columns, then determinant

    if checking:
        print('Checking determinants.')
        if not Dets[complex_embed_conj].imag() == 0:
            print('Determinant minus complex embeddings is not real!')
        for embedding in field.real_places: # determinant when remove column of real embedding
            m = get_index[embedding]
            conjDet = get_det(M, m, cconj_index) # delete m^th and n^th columns, then determinant
            if not abs(conjDet.conjugate() - Dets[embedding]) < 10^(-5):
                print('Determinants do not exhibit expected conjugacy!')
            sum_dets = conjDet + Dets[embedding]
            if not (sum_dets/Dets[complex_embed_conj] - (-1)^(n + m) < 10^(-5)):
                print('Sum of determinants does not give the expected sign!')
            product = Dets[complex_embed_conj]^(-1)*Dets[embedding]
            print('Product of determinant:', product.n(digits = 8))

        print()
        print('Computing (signed) regulators.')
        print('Remove both complex places:\t', Dets[complex_embed_conj].n(digits = 10))
        for embedding in field.real_places:
            m = get_index[embedding]
            conjDet = get_det(M, m, cconj_index)
            sum_dets = conjDet + Dets[embedding]
            print('Remove real place at m =', m, ':\t', sum_dets.n(digits = 10))
        print('\n')
   
    # compute Conjectural Stark unit
    log_Stark = scriptLs[complex_embed] + get_decimals(scriptRs[complex_embed])
    weighted_sum_Ls = 0
    for embedding in field.real_places: # for each real place
        m = get_index[embedding] # code version of the "m-1" in exponent of -1 (in Overleaf)
        weighted_sum_Ls += (-1)^(n + m)*Dets[embedding]*scriptLs[embedding]
    log_Stark += Dets[complex_embed_conj]^(-1)*weighted_sum_Ls

    return exp(log_Stark)

PARI code to work with number fields

In [17]:
# from https://ask.sagemath.org/question/57338/computing-ray-class-numbers/
from sage.groups.abelian_gps.abelian_group import AbelianGroup_class
from sage.groups.abelian_gps.abelian_group_element import AbelianGroupElement

class RayClassGroup(AbelianGroup_class):
    def __init__(self, number_field, mod_ideal = 1, mod_archimedean = None):
        if mod_archimedean == None:
            mod_archimedean = (1,) * len(number_field.real_places()) # default is to allow ramification at the real places

        bnf = gp(number_field.pari_bnf())
        # Use PARI to compute ray class group
        bnr = bnf.bnrinit([mod_ideal, mod_archimedean], 1)
        invariants = bnr[5][2]         # bnr.clgp.cyc
        invariants = tuple(ZZ(x) for x in invariants)

        super().__init__(invariants, names = 'f')
        self.__number_field = number_field
        self.__bnr = bnr

    def _element_constructor_(self, *args, **kwargs):
        if isinstance(args[0], AbelianGroupElement):
            return AbelianGroupElement(self, args[0])
        else:
            I = self.__number_field.ideal(*args, **kwargs)

            # Use PARI to compute class of given ideal
            g = self.__bnr.bnrisprincipal(I)[1]
            g = [ ZZ(x) for x in g ]
            return AbelianGroupElement(self, g)

In [18]:
# PARI CODE TO COMPUTE GENERATORS OF RAY CLASS GROUP
# some_irreducible_polynomial = x^3 - 2;
# bnf = bnfinit(some_irreducible_polynomial);
# f = modulus;
# bnr = bnrinit(bnf, f, flag = 1);
# generators = bnr.gen;
# # convert output to Sage?

In [19]:
def check_conjectural_Stark_unit(field, Stark):
    # determine candidate min_poly of Stark
    G = RayClassGroup(field.K, field.f)
    print('Order of ray class group =', G.order())
    max_degree = G.order()*field.n
    min_poly = algdep(Stark, max_degree) 
    print('Possible minimal polynomial:', min_poly)
    # check generates Abelian extension of K
    
    # check absolute value of (extended) complex embedding

In [20]:
def RS_cubic_example():
    """This is RS08 example 4.1.1, which is the same as Ren03 example 7.1.1."""
    x = PolynomialRing(RationalField(), 'x').gen()
    Stark_poly = x^18 - 7767*x^17 + 51550065*x^16 - 199524692622*x^15 + 520755985257966*x^14 - 1828056747902004*x^13 + 24870880029533226*x^12 - 80588629212013080*x^11 + 116076408275027511*x^10 - 118102314911180623*x^9 + 116076408275027511*x^8 - 80588629212013080*x^7 + 24870880029533226*x^6 - 1828056747902004*x^5 + 520755985257966*x^4 - 199524692622*x^3 + 51550065*x^2 - 7767*x + 1
    eta1 = 0.000176873039367043280880349386800159907 + 0.00013914485150266336187777561965299137901*I
    eta2 = -0.00011139315967699678393390174998 + 0.1530477331998331671189456454294012748596*I
    eta3 = 389.3671369204488907092470819163858452206092 + 5117.3074407766953210778035806417757009832205*I
    eta4 = 3492.3844689827497426203215698092503893499943 - 2747.4350679871860308834451055082268845522111*I
    eta5 = -0.0047555909249912080941723097768210037586 - 6.5339057910384706325415332807878168391893*I
    eta6 = 0.0000147832257430033251513884212903471304 - 0.0001942904367627936863506583832719023587*I
    etas = [eta1, eta2, eta3, eta4, eta5, eta6]
    roots_of_unity = []
    
    zero_cutoff = 10^(-5)
    roots = Stark_poly.roots(ring = QQbar) # we get correct coefficients if we use QQbar instead of CDF (cuz no imprecision)
    for root in roots:
        for j in range(0, len(etas)):
            eta = etas[j]
            if abs(eta - root[0]) < zero_cutoff:
                print('eta', j + 1)
            if abs(eta.conjugate() - root[0]) < 10^(-5):
                print('conjugate of eta', j + 1)
        if abs(abs(root[0]) - 1) < zero_cutoff:
            print('root of unity')
            roots_of_unity.append(root[0])
        # print(root[0])
    
    print('\n')
    
    max = 0
    product = 1
    for root in roots:
        product *= x - root[0].n(digits = 16) # increasing the number of digits decreases the differences
    print('Difference between conjectural coefficient and actual coefficient:')
    for pair in zip(product.coefficients(), Stark_poly.coefficients()):
        candidate = round(pair[0].real())
        if max < abs(candidate):
            max = abs(candidate)
        actual = pair[1]
        print(candidate - actual, end = ',')
    
    print('\n max coefficient:', max)

RUN FUNCTIONS

In [21]:
# domains = [RS_domain('4.1.1'), RS_domain('4.2.1'), Espinoza_domain('3.1.1'), Espinoza_domain('3.1.2'), Espinoza_domain('3.2.1')]

In [22]:
# for ex_domain in domains:
#     print(factor(ex_domain.field.f))

In [23]:
# for ex_domain in domains:
#     frakc = find_valid_c(ex_domain)
#     ell = norm(frakc)
#     print('n =', ex_domain.field.n)
#     print('ell =', ell)
#     print(factor(frakc))
#     print()

In [24]:
domain = RS_domain('4.2.1')

# RS 4.1.1 expected: 0.00000147 - 0.0001942*I
# on 2/22, RS 4.1.1 gave 2.25877401705118 + 0.418824655883687*I (2 min, recursive calculation of L's)
# on 2/23, RS 4.1.1 gave 2.25877401705118 + 0.418824655883687*I (1.25 min, changed my_log to log)
# on 2/23, RS 4.1.1 gave 2.25877401705118 + 0.418824655883687*I (11 seconds, less redundant Bern, no checking)
# on 3/3, RS 4.1.1 gave Stark = -1.48261021731930 - 1.75480514473317*I (with scriptR term in logStark)
# on 3/3, RS 4.1.1 gave Stark = -1.48261021730530 - 1.75480514473927*I (weighted average rotations)
# on 3/4, RS 4.1.1 gave Stark = -0.33680492092884506 - 2.674550295677089*I (Ruijsenaars logGamma, similar with weighted rotations)
# on 3/4, RS 4.1.1 gave Stark = -1.32004716104508 - 1.99575700722203*I ??? (rounding xvecs to 13)
# on 3/4, RS 4.1.1 gave Stark = -1.32004715896163 - 1.99575701022112*I ??? (rounding xvecs to 13, 10)
# on 3/5, RS 4.1.1 gave Stark = -0.336805816725196 - 2.67454258804582*I (rounding xvecs to 13)

# RS 4.2.1 expected: -0.0011 - 0.00039*I
# on 2/22, RS 4.2.1 gave -6.33156080996837 - 4.40571148278206*I (recursive calculation of L's)
# on 2/22, RS 4.2.1 gave Stark = 1.37260038321546 - 7.59044953529379*I (18 sec, changed my_log to log)
# on 3/3, RS 4.2.1 gave Stark = 1.37260038321546 - 7.59044953529379*I (with scriptR term in logStark)
# on 3/3, RS 4.2.1 gave Stark = 1.37260038241677 - 7.59044953455892*I (weighted average rotations)
# on 3/4, RS 4.2.1 gave Stark = 1.36996366388797 - 7.59079063379579*I (Ruijsenaars logGamma, similar with weighted rotations)
# on 3/4, RS 4.2.1 gave Stark = 1.38065955586508 - 7.58946322916398*I (rounding xvecs to 13)
# on 3/4, RS 4.2.1 gave Stark = 1.38065955380613 - 7.58946323020960*I (rounding xvecs to 13, 10)

domain = Espinoza_domain('3.1.1') 

# on 3/3, Espinoza 3.1.1 gave Stark = 0.0000459256583619805 - 0.0000206778947121869*I (2 min, no graphing in recursive log_gamma)
# on 3/3, Espinoza 3.1.1 gave Stark = 9.13946565484881e-6 + 0.0000495299060525255*I (2 min, with weighted average rotations)
# on 3/3, Espinoza 3.1.1 gave Stark = 0.0000195938586314831 - 0.0000463985143042219*I (with scriptR term in logStark)
# on 3/3, Espinoza 3.1.1 gave Stark = 0.0000195938588537891 - 0.0000463985142137758*I (with weighted average rotation)
# on 3/4, Espinoza 3.1.1 gave Stark = 0.0000195825168136005 - 0.0000463972152934633*I (Ruijsenaars logGamma, similar with weighted rotations)
# on 3/4, Espinoza 3.1.1 gave Stark = 0.0000195938860047196 - 0.0000463985042969064*I (rounding xvecs to 13)
# on 3/4, Espinoza 3.1.1 gave Stark = 0.0000195938848475992 - 0.0000463985047153235*I (rounding xvecs to 13, 10)

# on 2/19, Espinoza 3.1.2 gave Stark = 1.2548779162320576 + 0.42445150118101505*I (changed Minkowski_embed precision, 40 seconds)
# on 2/19, Espinoza 3.1.2 gave Stark = 1.25487766577591 + 0.424452249141138*I (recursive calculation of L's)
# on 2/23, Espinoza 3.1.2 gave Stark = 1.25487766670896 + 0.424452249118543*I
# on 2/23, Espinoza 3.1.2 gave Stark = 1.25487766670896 + 0.424452249118543*I (5 seconds, less redundant Bern, no checking)
# on 3/3, Espinoza 3.1.2 gave Stark = 1.25487766653224 + 0.424452248237850*I (with scriptR in logStark and any rotation)

# on 2/23, Espinoza 3.2.1 gave Stark = 1959.91183355683 - 2351.30777536057*I (2 hours with checking = True, less 50 min if not)
# on 3/3, Espinoza 3.2.1 gave Stark = 61.4220736533477 + 3060.41336728476*I (with scriptR term in logStark)
# on 3/3, Espinoza 3.2.1 gave Stark = 61.4313120610344 + 3060.40977499154*I (with weighted average rotation)

# R.<x> = PolynomialRing(ZZ)
# min_poly = x^2 + 2
# f = 2
# field = FieldData(min_poly, f)
# domain = compute_fundamental_domain(field)

domain.print_cones()

fraka = domain.field.K.ideal((1))

frakc = find_valid_c(domain)
ell = norm(frakc)
print('c is', frakc, 'with ell =', ell, '\n') 

The field with min_poly x^3 - x + 1 is ATR.
Positive S-units =  [-1, -theta]

Computing Shintani domain for ATR field of x^3 - x + 1 

Counts of nonzero orientations, listed -1,1: 	 [1, 0] 

The total number of cones is  1 

The field with min_poly x^3 + x^2 - 1 is ATR.
Positive S-units =  [-1, theta^2 + theta]
Using units = [-1, theta]

Computing Shintani domain for ATR field of x^3 + x^2 - 1 

Counts of nonzero orientations, listed -1,1: 	 [3, 1] 

The total number of cones is  4 

Printing cones.
Cone ((), 1, 1)  	 orientation =  -1
vertices =  [1, 2*theta^2 + theta, 2*theta^2 + 2*theta + 1]
boundaries =  [False, True, False]
Cone ((), 1, 2)  	 orientation =  -1
vertices =  [2*theta^2 + 2*theta + 1, theta, 2*theta + 1]
boundaries =  [False, True, False]
Cone ((), 2, 1)  	 orientation =  1
vertices =  [1, theta + 2, 2*theta^2 + theta]
boundaries =  [True, False, False]
Cone ((), 2, 2)  	 orientation =  -1
vertices =  [2*theta^2 + 2*theta + 1, 2*theta^2 + theta, theta]
boundaries =  [

In [25]:
domain.cones

{((), 1, 1): [1, 2*theta^2 + theta, 2*theta^2 + 2*theta + 1],
 ((), 1, 2): [2*theta^2 + 2*theta + 1, theta, 2*theta + 1],
 ((), 2, 1): [1, theta + 2, 2*theta^2 + theta],
 ((), 2, 2): [2*theta^2 + 2*theta + 1, 2*theta^2 + theta, theta]}

In [26]:
# check_embedding_LR(domain.field, scriptLs, scriptRs)

In [27]:
(scriptLs, scriptRs) = compute_scriptLsRs(domain, fraka, frakc, threshold = 10, M = 20, checking = True)

Computing the R_p sets.

Computing matrices of cones.
mu = ((), 1, 1) 	 rotation = e^(-16455/55829*I*pi) = 0.6010749932477549 - 0.7991926253990407*I
mu = ((), 1, 2) 	 rotation = e^(-12521/12907*I*pi) = -0.9955896378954601 - 0.0938150996118788*I
mu = ((), 2, 1) 	 rotation = I*e^(-68533/98731*I*pi) = 0.819702746166735 - 0.5727891478779196*I
mu = ((), 2, 2) 	 rotation = e^(-18417/20477*I*pi) = -0.950471685701309 - 0.3108111559775679*I

Getting xvecs for ideal = 1

cone_gens = [11, 22*theta^2 + 11*theta, 22*theta^2 + 22*theta + 11]
final count of xvec: 2662

cone_gens = [22*theta^2 + 22*theta + 11, 11*theta, 22*theta + 11]
final count of xvec: 2662

cone_gens = [11, 11*theta + 22, 22*theta^2 + 11*theta]
final count of xvec: 2662

cone_gens = [22*theta^2 + 22*theta + 11, 22*theta^2 + 11*theta, 11*theta]
final count of xvec: 2662

Getting xvecs for ideal = Fractional ideal (theta^2 - theta - 2)

cone_gens = [11, 22*theta^2 + 11*theta, 22*theta^2 + 22*theta + 11]
final count of xvec: 242

con

KeyboardInterrupt: 

In [ ]:
# Stark = compute_Stark_from_LRs(domain.field, scriptLs, scriptRs, checking = True)
# print('Stark =', Stark)